In [11]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
F:\anacondathree\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [12]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

32 documents
10 classes ['5g', 'delivery', 'goodbye', 'greeting', 'items', 'payments', 'phones', 'screensize', 'thanks', 'tv']
66 unique lemmatized words ["'s", '5g', 'a', 'about', 'accept', 'anyone', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'delivery', 'do', 'doe', 'get', 'good', 'goodbye', 'have', 'hd', 'hello', 'helpful', 'hey', 'hi', 'how', 'i', 'in', 'is', 'item', 'kind', 'later', 'latest', 'long', 'lot', 'mastercard', 'maximum', 'me', 'model', 'my', 'of', 'only', 'pay', 'paypal', 'phone', 'qled', 'samsung', 'screen', 'screensize', 'see', 'sell', 'shipping', 'size', 'take', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'tv', 'what', 'when', 'which', 'with', 'you']


In [13]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        # output is a '0' for each tag and '1' for current tag (for each pattern)
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-13-cd2d2c45d8a7>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [14]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=15, batch_size=100, verbose=1)
print("model created")

Epoch 1/15
2112/2112 [==============================] - 142s 67ms/step - loss: 2.2930 - accuracy: 0.1458
Epoch 2/15
2112/2112 [==============================] - 143s 68ms/step - loss: 2.0956 - accuracy: 0.2760
Epoch 3/15
2112/2112 [==============================] - 151s 72ms/step - loss: 1.8777 - accuracy: 0.4081
Epoch 4/15
2112/2112 [==============================] - 144s 68ms/step - loss: 1.6128 - accuracy: 0.5440
Epoch 5/15
2112/2112 [==============================] - 141s 67ms/step - loss: 1.3133 - accuracy: 0.6534
Epoch 6/15
2112/2112 [==============================] - 145s 69ms/step - loss: 1.0495 - accuracy: 0.7372
Epoch 7/15
2112/2112 [==============================] - 135s 64ms/step - loss: 0.8154 - accuracy: 0.8049
Epoch 8/15
2112/2112 [==============================] - 161s 76ms/step - loss: 0.6191 - accuracy: 0.8688
Epoch 9/15
2112/2112 [==============================] - 153s 72ms/step - loss: 0.5250 - accuracy: 0.8826
Epoch 10/15
2112/2112 [==============================] 

In [15]:
model.save('chatbot_model.h5', hist)

In [16]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))